# Sequía en Bolivia

El sistema Dewetra de la Fundación CIMA produce reportes del riesgo de desastres naturales dirigidos a [analistas del Ministerio de Defensa Civil](http://defensacivil.gob.bo/web/pagina/que-es-snatd.html). Si bien la mayoría de [las funciones de este sistema](http://bolivia.mydewetra.cimafoundation.org/wiki/intro.html) están reservadas para usuarios institucionales, algunos reportes son públicos. En particular, reportes del nivel de sequía pueden ser consultados mediante [un portal de senamhi](http://monitorsequias.senamhi.gob.bo). Estos reportes tienen 3 tipos de información:

- Mapas que clasifican el territorio nacional en 4 niveles de sequía
- Resúmenes del nivel de sequía por regiones
- Series de tiempo del porcentaje de territorio en cada uno de los 4 niveles a nivel nacional y por región

Este cuaderno es una demostración de cómo extraer esta información.

In [146]:
# dependencias

import datetime as dt
import json
import requests
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import locale
from IPython.display import display, Markdown
import re
plt.style.use('../0_common/estilo.mplstyle')
pd.options.display.max_colwidth=500
locale.setlocale(locale.LC_TIME, 'es_US.UTF8');

Una lista de boletines que contienen mapas y resúmenes puede ser consultada en [este índice](https://bolivia.mydewetra.cimafoundation.org/portal2_bulletin_prod/bulletininstance/?schema=10&is_published=true&is_open=false). Primero descargo el índice y filtro aquellas entradas que corresponden a boletines que contengan mapas.

In [20]:
INDEX_URL = 'https://bolivia.mydewetra.cimafoundation.org/portal2_bulletin_prod/bulletininstance/?format=json&is_open=false&is_published=true&schema=10'
index = requests.get(INDEX_URL)
bulletins = [i for i in index.json() if len([c for c in i['components'] if c['tag'] == "geojson_map"]) > 0]

Cada boletín corresponde a un mes:

In [93]:
sorted([i['date'] for i in bulletins])

['2020-06-01',
 '2020-07-01',
 '2020-08-02',
 '2020-09-01',
 '2020-10-01',
 '2020-11-01',
 '2020-12-01',
 '2021-01-01',
 '2021-02-01',
 '2021-03-01',
 '2021-04-01',
 '2021-05-01',
 '2021-06-01',
 '2021-07-01',
 '2021-08-01',
 '2021-09-01',
 '2021-10-01',
 '2021-11-10',
 '2021-12-01',
 '2022-02-01',
 '2022-03-01',
 '2022-04-01',
 '2022-05-01',
 '2022-06-01']

Para cada boletín, descargo todos los campos de resumen y el mapa en formato `geojson`, que guardo en el directorio `data/sequia`. Almaceno temporalmente esta información en la lista `all_data` que luego convertiré en un dataframe. 

In [96]:
GEOJSON_DIR = 'data/sequia'

In [97]:
def consume_bulletin(json_bulletin):
    mes = dt.datetime.strptime(json_bulletin['date'], '%Y-%m-%d')
    print(mes)
    data = {}
    fields = ['date', 'dateto', 'timestamp', 'last_modified']
    for field in fields:
        data[field] = json_bulletin[field]
    for component in json_bulletin['components']:
        if component['tag'] != 'geojson_map':
            tag = component['tag']
            text = requests.get(component['instance']).json()['text']
            data[tag] = text
        else:
            filename = '{}/{}.geojson'.format(GEOJSON_DIR, mes.strftime('%Y%m'))
            with open(filename, 'w+') as f:
                geojson = requests.get(component['instance']).json()
                json.dump(geojson, f, ensure_ascii=False)
                data['geojson'] = filename
    all_data.append(data)

In [18]:
all_data = []

In [21]:
for bulletin in bulletins:
    consume_bulletin(bulletin)

2021-02-01 00:00:00
2021-10-01 00:00:00
2021-05-01 00:00:00
2021-09-01 00:00:00
2020-06-01 00:00:00
2020-07-01 00:00:00
2020-08-02 00:00:00
2021-11-10 00:00:00
2021-03-01 00:00:00
2022-04-01 00:00:00
2020-09-01 00:00:00
2020-12-01 00:00:00
2020-11-01 00:00:00
2021-06-01 00:00:00
2021-12-01 00:00:00
2022-02-01 00:00:00
2021-01-01 00:00:00
2020-10-01 00:00:00
2021-04-01 00:00:00
2022-06-01 00:00:00
2021-07-01 00:00:00
2021-08-01 00:00:00
2022-03-01 00:00:00
2022-05-01 00:00:00


In [22]:
df = pd.DataFrame(all_data)

In [26]:
for col in ['date', 'dateto', 'timestamp', 'last_modified']:
    df[col] = pd.to_datetime(df[col])
df = df.sort_values('date')

In [101]:
display(Markdown('Existen {} boletines, cada uno con {} campos, de los cuales {} son textos de resumen. Un boletín tiene estos campos:'.format(
    df.shape[0],
    df.shape[1],
    len([col for col in df.columns if 'text_' in col])
)))
display(df.sample().T)

Existen 24 boletines, cada uno con 40 campos, de los cuales 35 son textos de resumen. Un boletín tiene estos campos:

,3
date,2021-09-01 00:00:00
dateto,2021-09-30 00:00:00
timestamp,2021-10-11 12:20:56.229544+00:00
last_modified,2021-10-15 22:12:10.687162+00:00
text_summary_general_SENAMHI,"En el mes de septiembre se observ&#243; un &#237;ndice combinado de sequ&#237;a entre moderado y d&#233;bil en gran parte de las macroregiones de Altiplano, Yungas-Chapare, Sabanas-Llanuras y Amazonia; un &#237;ndice combinado de sequ&#237;a desde d&#233;bil a extrema en gran parte de las macroregiones de la Chiquitania y Chaco; de manera puntual un &#237;ndice combinado de sequ&#237;a d&#233;bil en la macroregi&#243;n de los Valles.&#160;<div>En cuanto al d&#233;ficit de agua en el suelo (S..."
text_summary_general_VIDECI,"De acuerdo al análisis de vulnerabilidad y riesgo, prevalece la alerta nivel Naranja por sequía en las regiones Altiplano, Amazonia, Chaco, Chiquitania, LLanuras Sabanas y Yungas Chapare con eventos de sequía débil a moderada, septiembre presentó con relación al pasado mes un ligero incremento en índices de sequía en la región altiplano aunque persisten algunos municipios, se evidencio también un aumento en la intensidad de índices de sequía en la Amazonia, se incrementó el territorio suscep..."
text_summary_Amazonia_MMAyA,No se identifican situaciones de sequ&#237;a cr&#237;ticas en la macroregi&#243;n Amazonia para el sector de competencia.
text_summary_Amazonia_SENAMHI,"En el mes de septiembre se observ&#243; un &#237;ndice cambiando de sequ&#237;a entre d&#233;bil a moderado en gran parte de la macrorregi&#243;n y de manera puntual un &#237;ndice combinado de sequ&#237;a severa al Noreste.&#160;<br><span>En cuanto a d&#233;ficit de agua en el suelo (SWDI) las Cuencas con valores significativos desde seco a extremadamente seco son: Abun&#225;, Mamor&#233; (Cuenca alta y baja), Beni (cuenca baja y media). Todos con niveles muy bajos en r&#237;os principales ..."
text_summary_Chaco_SENAMHI,"En el mes de septiembre se observ&#243; un &#237;ndice cambiando de sequ&#237;a entre d&#233;bil en gran parte de la macroregion del Chaco y demanera puntual al Centro, Norte y Este de la macroregion.&#160;<div>En cuanto a d&#233;ficit de agua en el suelo (SWDI) las Cuencas que presentan valores significativos desde seco a extremadamente seco son: Parapet&#237; (cuenca baja), San Miguel y afluentes secundarios. Todos con niveles muy bajos en r&#237;os principales y afluentes secundarios.&#16..."
text_summary_Chaco_MMAyA,Cuencas con presencia de sequ&#237;a: Rio Quimome y R&#237;o San Ramon &#8211; Curupayty.


In [160]:
df.to_csv('data/boletines.csv', index=False)

Un mapa contiene una lista de polígonos, cada uno con 2 atributos: `cat` y `niv`. `niv` es el nivel de sequía, que puede ser un valor del 1 (débil) al 4 (extremo). Territorios sin una clasificación son considerados normales. A continuación visualizo los 4 últimos mapas en el sistema:

In [71]:
dep = gpd.read_file('../0_common/mapas/departamentos.geojson') # límites departamentales
background = '#c2c8cc' # el fondo de la figura

In [70]:
def plot_sequia(gdf, title, ax=None):
    
    note_color = '#686f73'
    if ax is None:
        f, ax = plt.subplots(1,1,figsize=(4,4), dpi=150)
        f.set_facecolor(background)
    gdf.plot(ax=ax, column='niv', cmap='OrRd')
    dep[dep.DESCRIPCIO == 'Departamento'].plot(ax=ax, facecolor="none", edgecolor=note_color, lw=.5)
    ax.set_axis_off()
    ax.annotate(title, xy=(.55, .85), xycoords='axes fraction', fontfamily='NYTImperial', fontstyle='italic', fontsize=15, color=note_color)

In [84]:
f, axs = plt.subplots(2,2,figsize=(8,8), dpi=150)
axs = axs.flatten()
plt.subplots_adjust(hspace=.01, wspace=.01)
f.set_facecolor(background)

for ax, (i, row) in zip(axs, df.iloc[-4:].iterrows()):
    gdf = gpd.read_file(row['geojson'])
    title = row['date'].strftime('%B')
    plot_sequia(gdf, title, ax=ax)
    
filename = 'plots/sequia2022.png'
f.savefig(filename, bbox_inches='tight', pad_inches=.2, dpi=150)
plt.close()
display(Markdown("![]({})".format(filename)))

![](plots/sequia2022.png)

[Los mapas deslizantes en el sitio oficial](http://monitorsequias.senamhi.gob.bo/#/maps/slider) son mucho más ilustrativos.

Finalmente, las series de tiempo están listadas en [un directorio apache](https://bolivia.mydewetra.cimafoundation.org/drought_public/MSBI/). Vienen en 3 formatos: `csv` en decimales, `csv` en porcentajes y `json` en decimales, para cada región y a nivel nacional. Tomo los `csv` en decimales:

In [123]:
APACHE_INDEX = 'https://bolivia.mydewetra.cimafoundation.org/drought_public/MSBI'
apache = pd.read_html(APACHE_INDEX)[0]
csvs = apache[(apache.Name.notna()) & (apache.Name.str.contains('last.csv'))].Name.tolist()

In [124]:
csvs

['MSBI_Altiplano_last.csv',
 'MSBI_Altiplano_perc_last.csv',
 'MSBI_Amazonia_last.csv',
 'MSBI_Amazonia_perc_last.csv',
 'MSBI_Bolivia_last.csv',
 'MSBI_Bolivia_perc_last.csv',
 'MSBI_Chaco_last.csv',
 'MSBI_Chaco_perc_last.csv',
 'MSBI_Chiquitania_last.csv',
 'MSBI_Chiquitania_perc_last.csv',
 'MSBI_Llanuras_Sabanas_last.csv',
 'MSBI_Llanuras_Sabanas_perc_last.csv',
 'MSBI_Valles_last.csv',
 'MSBI_Valles_perc_last.csv',
 'MSBI_Yungas_Chapare_last.csv',
 'MSBI_Yungas_Chapare_perc_last.csv']

Cada csv tiene una columna para la fecha y otras cinco para niveles de sequía, incluyendo un nivel `0` para donde la situación es normal. Cada valor es el porcentaje de territorio que clasifica dentro de cada nivel de sequía en el periodo de reporte. Ésta es la forma de una tabla:

In [158]:
pd.read_csv('{}/{}'.format(APACHE_INDEX, csvs[0]))

,date,LEV_0,LEV_1,LEV_2,LEV_3,LEV_4
0,2007-02,0.53,0.34,0.13,0.01,0.0
1,2007-03,0.86,0.13,0.01,0.00,0.0
2,2007-04,0.79,0.19,0.02,0.00,0.0
3,2007-05,0.94,0.06,0.00,0.00,0.0
4,2007-06,0.77,0.21,0.02,0.00,0.0
...,...,...,...,...,...,...
179,2022-01,0.71,0.28,0.01,0.00,0.0
180,2022-02,0.78,0.21,0.00,0.00,0.0
181,2022-03,0.51,0.43,0.06,0.00,0.0
182,2022-04,0.41,0.52,0.07,0.00,0.0


La forma más concisa sería una tabla con 4 columnas: `fecha`, `región`, `nivel` y `porcentaje`. Costruyo esta tabla con todas las series de tiempo disponibles:

In [155]:
timeseries = []
for csv in csvs:
    name = re.findall('MSBI_(.*)_last.csv', csv)[0]
    timeserie = pd.read_csv('{}/{}'.format(APACHE_INDEX, csv), index_col=['date'])
    timeserie = timeserie.stack().reset_index(name='porcentaje')
    timeserie.insert(0, 'region', name)
    timeseries.append(timeserie)
timeseries = pd.concat(timeseries)
timeseries.columns = ['region', 'date', 'nivel', 'porcentaje']
timeseries['date'] = pd.to_datetime(timeseries['date'])
timeseries['nivel'] = timeseries['nivel'].apply(lambda x: int(x.split('_')[-1]))

In [156]:
timeseries

,region,date,nivel,porcentaje
0,Altiplano,2007-02-01,0,0.53
1,Altiplano,2007-02-01,1,0.34
2,Altiplano,2007-02-01,2,0.13
3,Altiplano,2007-02-01,3,0.01
4,Altiplano,2007-02-01,4,0.0
...,...,...,...,...
915,Yungas_Chapare_perc,2022-05-01,0,23.8%
916,Yungas_Chapare_perc,2022-05-01,1,43.8%
917,Yungas_Chapare_perc,2022-05-01,2,30.4%
918,Yungas_Chapare_perc,2022-05-01,3,1.9%


In [159]:
timeseries.to_csv('data/timeseries.csv', index=False)